In [2]:
class Decoder:
    '''
    Processes the next bit
    '''
    def process_next_bit(self, bit):
        pass

    '''
    Outputs the final data as a string
    '''
    def finalize(self) -> str:
        pass

In [3]:
from enum import IntEnum
import math

In [141]:
class BeatDecoder:
    def __init__(self, initial_beat_dur = 1, cal_req = 5,  beat_error = 0.15, relearn_factor = 0.5, learning_factor = 0.5):
        self.beat_duration = initial_beat_dur
        self.num_of_bits = 0
        self.bit = 0
        self.calibrated = False
        self.beat_error = beat_error
        self.closely_calibrated = 0
        self.calibrate_goal = cal_req
        self.relearn_factor = relearn_factor
        self.learning_factor = learning_factor

    def get_beat(self):
        floored = math.floor(self.beat_duration)
        return floored if (self.beat_duration - floored) < 0.5 else math.ceil(self.beat_duration)

    def has_calibrated(self) -> bool:
        return self.calibrated

    '''
    Calibrates the beat decoder and then outputs that to the hardware
    '''
    def calibrate_beat(self, bit) -> bool:
        if bit == self.bit:
            self.num_of_bits += 1
        else:
            # offset = min(map(lambda dur: abs(self.num_of_bits - self.beat_duration * dur), [1,3,7])) / self.beat_duration - 1

            offset = None
            for num_beats in [7, 3, 1]:
                offset = (self.num_of_bits - self.beat_duration * num_beats) / self.beat_duration

                if abs(offset) < 1:
                    break

            # offset = (self.num_of_bits % self.beat_duration) / self.beat_duration - 1
            if abs(offset) > self.beat_error:
                # self.beat_duration = max(self.beat_duration + (1 + offset * self.learning_factor), 1)
                self.beat_duration = max((self.beat_duration + self.num_of_bits) / 2, 1)
                self.closely_calibrated = self.closely_calibrated * self.relearn_factor

                if self.calibrated:
                    self.calibrated = False
            else:
                self.beat_duration *= 1 + (offset * math.log(self.calibrate_goal - self.closely_calibrated + 1, self.calibrate_goal))
                if not self.calibrated:
                    self.closely_calibrated += 1

                if not self.calibrated and self.closely_calibrated >= self.calibrate_goal:
                    self.calibrated = True

            self.bit = bit
            self.num_of_bits = 0

            print('offset:', offset)
            print('beat_duration:', self.beat_duration)

In [143]:
'''
Test the beat decoder
'''
import random
bit_string = '''10111000 111010101000 11101011101000 1110101000
1000 101011101000 111011101000 1010101000 101000
1011101110111000 111010111000 101110101000 1110111000
11101000 11101110111000 10111011101000 1110111010111000
1011101000 10101000 111000 1010111000 101010111000
101110111000 11101010111000 1110101110111000 11101110101000'''
bit_string2 = '''101110000000 1110101010000000 111010111010000000 11101010000000
10000000 1010111010000000 1110111010000000 10101010000000 1010000000
10111011101110000000 1110101110000000 1011101010000000 11101110000000
111010000000 111011101110000000 101110111010000000 11101110101110000000
10111010000000 101010000000 1110000000 10101110000000 1010101110000000
1011101110000000 111010101110000000 11101011101110000000 111011101010000000'''
bit_string3 = bit_string * 100
beat_duration = 15

lose_factor = 0.1
# switch_factor = 0.1
double_factor = 0.1
# lose_factor = 0
# switch_factor = 0
# double_factor = 0
finished = True
correct = True

try:
    while finished and correct:
        bits = list()
        # print('bit samples:', len(bits))
        for letter in bit_string3.split():
            for bit in letter:
                bit = int(bit)
                for times in range(beat_duration):
                    noise = random.random()
                    if noise < lose_factor:
                        pass
                    elif noise < (lose_factor + double_factor):
                        bits.append(bit)
                        bits.append(bit)
                    else:
                        bits.append(bit)
        print("actual beat_duration is", beat_duration)
        # decoder = BeatDecoder(cal_req= 5, beat_error=0.30, relearn_factor=0.65, learning_factor = 0.8)
        decoder = BeatDecoder(cal_req= 7, beat_error=0.3, relearn_factor=0.6, learning_factor = 0.9)
        bit_index = 0
        while bit_index < len(bits) and not decoder.has_calibrated():
            decoder.calibrate_beat(bits[bit_index])
            # print("calibrating with ", bits[bit_index])
            bit_index += 1

        finished = bit_index < len(bits)
        output_text = 'ran out of bits. final beat decoding:' if not finished else 'calibrated bits are:'
        correct = beat_duration == decoder.get_beat()
        print(output_text, decoder.get_beat())

        if finished and correct:
            beat_duration += 1 

    print("final accurate beat duration:", beat_duration - 1)
except KeyboardInterrupt:
    print("stopped at beat duration:", beat_duration - 1)


actual beat_duration is 15
offset: -1.0
beat_duration: 1
offset: 10.0
beat_duration: 6.0
offset: -0.8333333333333334
beat_duration: 9.5
offset: 3.210526315789474
beat_duration: 24.75
offset: -0.9393939393939394
beat_duration: 37.875
offset: 0.24092409240924093
beat_duration: 47.626171747082104
offset: -0.685047119057622
beat_duration: 31.313085873541052
offset: -0.6167736374350837
beat_duration: 21.656542936770528
offset: -0.3997195195024701
beat_duration: 17.328271468385264
offset: -0.24978091301732094
beat_duration: 12.763868775284775
offset: 0.25353842801811655
beat_duration: 15.947874843096423
offset: -0.24754864719830647
beat_duration: 12.387126000062358
offset: 0.3098880240496145
beat_duration: 26.69356300003118
offset: 0.8731107570743403
beat_duration: 38.34678150001559
offset: -0.582755074242841
beat_duration: 27.173390750007794
offset: -0.5215908047840415
beat_duration: 20.0866953750039
offset: -0.30302124174085143
beat_duration: 17.04334768750195
offset: -0.5356954062024444
b

In [137]:
class SWDecoder(Decoder):
    # ------------------------------------------------------
    # Letter State Machine
    # ------------------------------------------------------
    def __init__(self, letters = "ETIANMSURWDKGOHVF L PJBXCYZQ"):
        self.letters = letters
        self.current_letter = 0
        self.beat_error_range = 0.15

        self.beat_duration = 1 # How many bits is a beat?
        self.prev_bit = 0 # The value of the previous bit
        self.num_of_bit = 0 # The number of occurrences of "prev bit"
        self.buffer = list()

    '''
    Initializes the machine
    '''
    def init_letters(self):
        self.current_letter = 0

    '''
    Shifts the state machine to the next letter

    Params:
    - is_dash(bool): is really some bit indiciating if the incoming symbol is a dot or dash
    '''
    def shift_letter(self, is_dash: bool) -> None:
        self.current_letter = self.current_letter * 2 + is_dash + 1

    '''
    Gets the letter that the current state machine is on

    Returns:
    The length 1 string that has the character
    None, if CURRENT_LETTER is not in the range of the list of letters
    '''
    def get_letter(self) -> str: 
        ret_letter = ""

        if self.current_letter > 0 and self.current_letter <= len(self.letters):
            ret_letter = self.letters[self.current_letter - 1]

        self.current_letter = 0

        return ret_letter

    '''
    Finishes the machine and grabs the last letter
    '''
    def finalize(self) -> str:
        return self.get_letter()
    

    '''
    A list of the possible parsings from the morse code
    '''
    class MEANING(IntEnum):
        # Symbol
        DOT = 0
        DASH = 1

        # Pauses
        NEXT_SYMBOL = 2
        NEXT_LETTER = 3
        NEXT_WORD = 4

        # Unknown
        UNKNOWN = -1

    '''
    Removes noise from the beat_duration

    Params:
    - beat_nums(float): the rough number of beats
    '''
    def remove_noise(self, beat_nums: float):
        error_ranges = [(dur, dur * (1 - self.beat_error_range), dur * (1 + (self.beat_error_range if dur != 7 else 1))) for dur in (1,3,7)]

        for (duration, duration_min, duration_max) in error_ranges:
            if beat_nums >= duration_min and beat_nums <= duration_max:
                return duration

        return -1


    '''
    Processes some parsed information and interacts with the "letter" state machine

    Params:
    - meaning (MEANING): some parsed info from the bitstream input

    Returns:
    Either a letter, a letter + a space, or None
    '''
    def process(self, meaning: MEANING) -> str | None:
        ret_proc = None
        match meaning:
            case SWDecoder.MEANING.DOT:
                self.shift_letter(SWDecoder.MEANING.DOT)
            case SWDecoder.MEANING.DASH:
                self.shift_letter(SWDecoder.MEANING.DASH)
            case SWDecoder.MEANING.NEXT_LETTER:
                ret_proc = self.get_letter()
            case SWDecoder.MEANING.NEXT_WORD:
                ret_proc = self.get_letter() + " "

        return ret_proc
    
    # ------------------------------------------------------
    # Beat Decoder
    # ------------------------------------------------------
    '''
    Determines what the last string of bit B means

    Returns:
    The information as one of the discrete meanings above.
    '''
    def parse_prev_inputs(self):
        beats = self.remove_noise(self.num_of_bit / self.beat_duration)

        meaning = None
        match (self.prev_bit, beats):
            case (1, 1):
                meaning = SWDecoder.MEANING.DOT
            case (1, 3):
                meaning = SWDecoder.MEANING.DASH
            case (0, 1):
                meaning = SWDecoder.MEANING.NEXT_SYMBOL
            case (0, 3):
                meaning = SWDecoder.MEANING.NEXT_LETTER
            case (0, 7):
                meaning = SWDecoder.MEANING.NEXT_WORD
            case _:
                meaning = SWDecoder.MEANING.UNKNOWN
        return meaning

    '''
    Processes the next bit

    Params:
    - bit(int): either a 0 or 1 that is either an "on" or "off" signal from the sender

    Returns:
    A letter if we have reached the end of a letter/word, else None
    '''
    def process_next_bit(self, bit: int) -> str | None:
        if bit == self.prev_bit:
            self.num_of_bit += 1
        else:
            meaning = self.parse_prev_inputs()
            maybe_output = self.process(meaning)

            self.prev_bit = bit
            self.num_of_bit = 1

            return maybe_output

NameError: name 'Decoder' is not defined

In [4]:
import sys

In [6]:
from pynq import Overlay, DefaultIP, allocate
import numpy as np

In [7]:
overlay = Overlay('/home/xilinx/pynq/overlays/project/decoder/process_next_bit.bit')

In [8]:
class HWDecoder(Decoder): # "/home/xilinx/pynq/overlays/project/decoder/process_next_bit.bit"
    # TODO: appends the list if a NEXT LETTEr
    # TODO: processes and the appends a " " if a NEXT WORD
#     class LetterInterpreter:
#         def __init__(self):
#             self.bit = 0
#             self.num_of_bits = 0
            
# #         def 
    
    def __init__(self, overlay, stream_buf_size = 1000) -> None:
        self.overlay = overlay
        self.send_channel = self.overlay.processNextBit.axi_dma.sendchannel
        self.recv_channel = self.overlay.processNextBit.axi_dma.recvchannel

        self.stream_buf_size = stream_buf_size
        self.buffer = list()
        self.next_letter = list()
        self.beat_duration = 0

    def init_letters(self):
        self.next_letter = list()
        
#     def 
        
    def process(self, in_bits, recv_chars):
        # decoder.start()
        # TODO: pass in the beat_duration
        self.send_channel.start()
        self.recv_channel.start()
        self.send_channel.transfer(in_bits)
        self.send_channel.wait()
        self.recv_channel.transfer(recv_chars)
        self.recv_channel.wait()
        
        return recv_chars

    def adjust_factor(self):
        pass

    def det_beat_dur(self):
        pass

    def process_next_bit(self, bit, finalize = False):
        if finalize or len(self.buffer) == self.stream_buf_size:
            list_capacity = len(self.buffer)
            recv_capacity = list_capacity #// 4
            in_bits = allocate(shape=(list_capacity,), dtype=np.uint32)
            recv_chars = allocate(shape=(recv_capacity,), dtype=np.uint32)
            
            in_bits[:] = self.buffer[:]
            print("in_bits", in_bits)
            output = self.process(in_bits, recv_chars)
            print("output", output)
            
            a_offset = ord('a')
            letter_list = [(chr(letter+a_offset-1) if letter else ' ') for letter in recv_chars if letter != 31]
            return_letter = ''.join(letter_list)
            self.buffer = []
            del in_bits, recv_chars
            
            return return_letter
        else:
            # TODO: what to do about the 100th thing being in the middle of letters
            self.buffer.append(bit)

    def finalize(self):
        return self.process_next_bit(0, finalize = True)

In [173]:
from pydub import AudioSegment
import numpy as np
import scipy.signal as signal
from scipy.signal import iirfilter, freqz,butter,filtfilt
import math
# import matplotlib.pyplot as plt

In [174]:
class FFT:
    def __init__(self, signal_data: list, sample_rate: int) -> None:
        self.signal_data = signal_data
        self.sample_rate = sample_rate

    def transform(self) -> tuple[float, float, float]:
        fft_result = np.fft.fft(self.signal_data)
        fft_magnitude = np.abs(fft_result)
        freqs = np.fft.fftfreq(len(self.signal_data), 1/self.sample_rate)

        max_freq_index = np.argmax(fft_magnitude)
        self.max_freq = freqs[max_freq_index]

        theshold = 0.1 * self.max_freq

        lower_bound = np.where(fft_result[:max_freq_index] < theshold)[0][-1]
        upper_bound = np.where(fft_result[max_freq_index+1:] > theshold)[0][0] + max_freq_index + 1

        # print(lower_bound, upper_bound)

        self.bandwidth = 300
        
        # print(freqs[lower_bound], freqs[upper_bound])
        
        return self.sample_rate, self.max_freq, self.bandwidth


In [175]:
class Filter:
    def filter(self, unfiltered: list, sample_rate: int):
        pass

In [176]:
class SWAudioFilter:
    def __init__(self, fft: FFT = FFT) -> None:
        self.fft = fft

    def gen_coef(self, num_coef: int, sample_rate: int, max_freq: float, bandwidth: float) -> list:
        nyquist_freq = sample_rate/2.0
        lower_cutoff = (max_freq - bandwidth / 2)/nyquist_freq
        upper_cutoff = (max_freq + bandwidth / 2)/nyquist_freq
        

        coefficients = list()

        for i in range(int(num_coef)):
            t = (i - (num_coef - 1) / 2) / sample_rate

            next_coef = None
            if i == (num_coef - 1) / 2:
                next_coef = 2 * math.pi * bandwidth
            else:
                next_coef = (math.sin(2 * math.pi * upper_cutoff * t) 
                             - math.sin(2 * math.pi * lower_cutoff * t)) / (math.pi * t)
                
            coefficients.append(next_coef)
       

        return coefficients

    
    def filter(self, unfiltered: list, sample_rate: int):
        
        fft = self.fft(unfiltered, sample_rate)
        sampling_rate, max_freq, bandwidth = fft.transform()
        lower_cutoff = (max_freq - bandwidth / 2)
        upper_cutoff = (max_freq + bandwidth / 2)
        print(sampling_rate, max_freq, bandwidth)
        

        # Bandwidth/ sampling_rate = num_of_coefs?
        # Bandwidth is pretty specific, so maybe keep it at a hard like 11?
        num_of_coefs = 200
        coefficients = self.gen_coef(num_of_coefs, sampling_rate, max_freq, bandwidth)
        
        


        print(coefficients)

        shift_reg = [0] * num_of_coefs
        filtered = list()
        accumulator = 0
        data = 0

        # TODO: finish filter
        for unfil_data in unfiltered:
            for i in range(num_of_coefs - 1, 0, -1):
                if i == 0:
                    shift_reg[0] = unfil_data
                    data = unfil_data
                else:
                    shift_reg[i] = shift_reg[i-1]
                    data = shift_reg[i]

                accumulator += data * coefficients[i]

            filtered.append(accumulator)

        return np.array(filtered)
    
    

In [177]:
class SWAudioFilter_test:
    def __init__(self, fft: FFT = FFT) -> None:
        self.fft = fft

    #def design_morse_bandpass_filter(self, sample_rate):
        # Define frequency range for Morse code signals (approximate)
        #low_freq = 300  # Frequency for dots
        #high_freq = 525  # Frequency for dashes
        #bandwidth = 300  # Adjust as needed

        # Normalize frequencies
        #low_cutoff = low_freq / (sample_rate / 2)
        #high_cutoff = high_freq / (sample_rate / 2)

        # Design a bandpass filter
        #b, a = butter(4, [low_cutoff, high_cutoff], btype='band')
        #return b, a

    #def filter(self, unfiltered: list, sample_rate: int):
        #fft = self.fft(unfiltered, sample_rate)
        #sampling_rate, max_freq, bandwidth = fft.transform()
        #print(max_freq)
        #lower_cutoff = (max_freq - bandwidth / 2)
        #pper_cutoff = (max_freq + bandwidth / 2)
        #b, a = self.design_morse_bandpass_filter(sample_rate)
        #filtered = filtfilt(b, a, unfiltered)
        #return filtered
    
    def generate_bandpass_filter(self,fs, lowcut, highcut, num_taps):
        nyquist = 0.5 * fs
        low = lowcut / nyquist
        high = highcut / nyquist

        taps = np.zeros(num_taps)
        for i in range(num_taps):
            if i - num_taps / 2 == 0:
                taps[i] = 2 * (high - low)
            else:
                taps[i] = (np.sin(2 * np.pi * high * (i - num_taps / 2)) - np.sin(2 * np.pi * low * (i - num_taps / 2))) / (np.pi * (i - num_taps / 2))
    
        return taps

    def apply_filter(self,signal, filter_taps):
        num_taps = len(filter_taps)
        filtered_signal = np.convolve(signal, filter_taps, mode='same')
        return filtered_signal

In [180]:
audio = AudioSegment.from_file('sos.mp3')
sample_rate = audio.frame_rate
signal_data = np.array(audio.get_array_of_samples())

filter = SWAudioFilter_test(FFT)
#filtered = filter.filter(signal_data, sample_rate)

# Sample rate and desired cutoff frequencies (in Hz)  # Sample rate
lowcut = 300.0  # Low cutoff frequency
highcut = 500.0  # High cutoff frequency

# Generate bandpass filter coefficients
num_taps = 4  # Filter length
t = np.linspace(0, 1, int(sample_rate), endpoint=False)
filter_taps = filter.generate_bandpass_filter(sample_rate, lowcut, highcut, num_taps)

# Apply bandpass filter
filtered_data = filter.apply_filter(signal_data, filter_taps)





from scipy.io.wavfile import write
write('filtered_sos.wav', sample_rate, filtered_data)

794624


In [5]:
# ------------------------------------------------------
# Translation
# ------------------------------------------------------
def translate(input_file, out = None, decoder_cls: Decoder = SWDecoder):
    if out is None:
        stdout = sys.stdout
    else:
        stdout = open(out, "w")

    decoder = decoder_cls()

    with open(input_file, "r") as in_file:
        decoder.init_letters()
        for line in in_file:
            for bit in line:
                if bit in [" ", "\n"]:
                    continue

                bit = int(bit)
                
                letter = decoder.process_next_bit(bit)

                if letter is not None:
                    print(letter, end = "", file = stdout)
                    
        print(decoder.finalize(), file = stdout)

    if out is not None:
        stdout.close()

In [6]:
from pathlib import Path
import time

In [7]:
'''
Basic Test Template to make other tests out of
'''
def translate_test(in_file, golden_file, result_file = None, time_it = False, **kwargs):
    if result_file is None:
        dot_split = in_file.split(".")
        result_file = "{}_out.{}".format("".join(dot_split[:-1]), dot_split[-1])
    
    timing = time.time()

    translate(in_file, out=result_file, **kwargs)

    if time_it:
        timing = time.time() - timing
    else:
        timing = 0

    correct_output = True

    with open(result_file, "r") as results:
        with open(golden_file, "r") as golden:
            next_result = results.readline().strip().lower()
            next_golden = golden.readline().strip().lower()

            while next_result != "" and next_golden != "":
                if next_result != next_golden:
                    print(f"'{next_result}' does not match '{next_golden}'")
                    correct_output = False
                    break
                next_result = results.readline()
                next_golden = golden.readline()

            if next_result != next_golden:
                print(f"One was empty:\n'{next_result}' does not match '{next_golden}'")
                correct_output = False
            
#     Path(result_file).unlink()

    if correct_output:
        timing_statement = "" if not time_it else f" in {timing:4.2} seconds"
        print(f"Successfully translated {in_file}{timing_statement}")

    return correct_output
                 


In [8]:
'''
Tests if the alphabet with no spaces is accurate
'''
def test1():
    translate_test("decoder_tests/test1.txt", "decoder_tests/golden_out1.txt", time_it = True)
    
test1()

Successfully translated decoder_tests/test1.txt in 0.0046 seconds


In [9]:
'''
Tests if the alphabet with 1 space is accuate
'''
def test2():
    translate_test("decoder_tests/test2.txt", "decoder_tests/golden_out2_3.txt", time_it = True)

test2()

Successfully translated decoder_tests/test2.txt in 0.002 seconds


In [10]:
'''
Tests if multiple spaces are ommitted
'''
def test3():
    translate_test("decoder_tests/test3.txt", "decoder_tests/golden_out2_3.txt", time_it = True)

test3()

Successfully translated decoder_tests/test3.txt in 0.002 seconds


In [11]:
'''
Tests the "the quick brown fox jumps over the lazy dog"
'''
def test4():
    translate_test("decoder_tests/test4.txt", "decoder_tests/golden_out4.txt", time_it = True)

test4()

Successfully translated decoder_tests/test4.txt in 0.0057 seconds


In [79]:
def testA_hw():
    translate_test("decoder_tests/untitled.txt", "decoder_tests/untitled1.txt", time_it = True, decoder_cls = HWDecoder)
testA_hw()

[1, 0, 1, 1, 1, 0, 0, 0]
in_bits [1 0 1 1 1 0 0 0]
output [0 0 0 0 0 0 0 0]
'        ' does not match 'a'
One was empty:
'        ' does not match 'a'


In [80]:
def test1_hw():
    translate_test("decoder_tests/test1.txt", "decoder_tests/golden_out1.txt", time_it = True, decoder_cls = HWDecoder)
    
test1_hw()

[1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1]
in_bits [1 0 1 1 1 0 0 0 1 1 1 0 1 0 1 0 1 0 0 0 1 1 1 0 1 0 1 1 1 0 1 0 0 0 1 1 1
 0 1 0 1 0 0 0 1 0 0 0 1 0 1 0 1 1 1 0 1 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 1 0
 1 0 1 0 1 0 0 0 1 0 1 0 0 0 1 0 1 1 1 0 1 1 1 0 1 1]
output [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1,

In [ ]:
def test2_2hw():
    translate_test("decoder_tests/test2ab.txt", "decoder_tests/golden_out2ab.txt", time_it = True, decoder = HWDecoder(overlay))
    
test2_2hw()

in_bits [1 0 1 1 1 0 0 0 1 1 1 0 1 0 1 0 1 0 0 0 0 0 0 0 1 1 1 0 1 0 1 1 1 0 1 0 0
 0 1 1 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 1 1 0 1 0 0 0 0 0
 0 0 1 1 1 0 1 1 1 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 1 0 1 0 0
 0 0 0 0 0 1 0 1 1 1 0 1 1 1 0 1 1 1 0 0 0 0 0 0 0 1 1 1 0 1 0 1 1 1 0 0 0
 0 0 0 0 1 0 1 1 1 0 1 0 1 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0 0 0 0 0 0 0 1 1 1
 0 1 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0 1 1 1 0 0 0 0 0 0 0 1 0 1 1 1 0 1 1 1 0
 1 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0 1 0 1 1 1 0 0 0 0 0 0 0 1 0 1 1 1 0 1 0 0
 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 1 0 1 1 1 0 0 0
 0 0 0 0 1 0 1 0 1 0 1 1 1 0 0 0 0 0 0 0 1 0 1 1 1 0 1 1 1 0 0 0 0 0 0 0 1
 1 1 0 1 0 1 0 1 1 1 0 0 0 0 0 0 0 1 1 1 0 1 0 1 1 1 0 1 1 1 0 0 0 0 0 0 0
 1 1 1 0 1 1 1 0 1 0 1 0 0 0 0 0 0 0]
output [31  1 31 31  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

In [ ]:
def test2_hw():
    translate_test("decoder_tests/test2.txt", "decoder_tests/golden_out2_3.txt", time_it = True, decoder = HWDecoder(overlay))
    
test2_hw()

In [ ]:
def test3_hw():
    translate_test("decoder_tests/test3.txt", "decoder_tests/golden_out2_3.txt", time_it = True, decoder = HWDecoder(overlay))
    
test3_hw()

In [ ]:
def test4_hw():
    translate_test("decoder_tests/test4.txt", "decoder_tests/golden_out4.txt", time_it = True, decoder = HWDecoder(overlay))
    
test4_hw()